In [1]:
from parseBRENDA import BRENDA
workDir = 'C:/Users/tinta/OneDrive/Documents/Projects/BRENDA'
dataFile = workDir + '/brenda_download.txt'

In [2]:
b = 1
id(b)

140737486680320

In [1]:
a = 1
id(a)

140737486680320

In [3]:
c = 2.1234
id(c)

140737486680352

In [4]:
c = 2.1234
id(c)

1440723102672

In [2]:
brenda = BRENDA(dataFile)
print(f'There are a total of {len(brenda.reactions)} reactions in BRENDA')

There are a total of 7558 reactions in BRENDA


In [5]:
r = brenda.reactions.get_by_id('1.1.1.1')
r

Enzyme identifier,1.1.1.1
Memory address,0x02f0022e19c8
Name,alcohol dehydrogenase
Systematic name,alcohol:NAD+ oxidoreductase
Reaction type,redox reaction
Mechanism,"a primary alcohol + NAD+ <=> an aldehyde + NADH + H+ (#4,41# ordered"


In [5]:
r.getSpecies()

{'1': {'name': 'Mus musculus',
  'proteinID': '',
  'refs': ['6', '7', '8', '11', '19', '22', '25', '38', '40', '41']},
 '2': {'name': 'Homo sapiens',
  'proteinID': '',
  'refs': ['3',
   '4',
   '5',
   '9',
   '10',
   '12',
   '13',
   '14',
   '19',
   '20',
   '21',
   '23',
   '24',
   '26',
   '27',
   '29',
   '30',
   '31',
   '32',
   '33',
   '34',
   '35',
   '36',
   '37',
   '3940',
   '42']},
 '3': {'name': 'Rattus norvegicus', 'proteinID': '', 'refs': ['1', '7', '22']},
 '4': {'name': 'Sus scrofa', 'proteinID': '', 'refs': ['2']},
 '5': {'name': 'Oryctolagus cuniculus', 'proteinID': '', 'refs': ['40']},
 '6': {'name': 'Homo sapiens',
  'proteinID': 'P40261 SwissProt',
  'refs': ['15', '16', '17', '18', '28']}}

In [4]:
r.reaction_type

'keto group transfer'

In [5]:
temp = r.getTemperatureData()

In [7]:
[l['value'] for l in temp['optimum']]

[40.0, 62.0, 37.0, 65.0, 65.0, 30.0, 45.0, 25.0, 20.0]

In [8]:
[l['value'] for l in temp['stability']]

[71.0, 67.5, 65.0, 60.0, 58.3, -999.0, 55.0, 40.0, 20.0, 50.0]

In [9]:
[l['value'] for l in temp['range']]

[[40.0, 70.0]]

In [8]:
r.getMetals().keys()

dict_keys(['Mg2+', 'Mn2+', 'Cu2+', 'Co2+', 'Ca2+'])

In [4]:
r.getSubstratesAndProducts()    

[{'substrates': ['D-ribose5-phosphate', 'D-xylulose 5-phosphate'],
  'products': ['D-glyceraldehyde 3-phosphate', 'sedoheptulose 7-phosphate']},
 {'substrates': ['D-fructose 6-phosphate', 'D-glyceraldehyde 3-phosphate'],
  'products': ['D-xylulose 5-phosphate', 'erythrose 4-phosphate']},
 {'substrates': ['D-glyceraldehyde 3-phosphate', 'glyceraldehyde 3-phosphate'],
  'products': ['D-xylulose 5-phosphate', 'D-xylulose 5-phosphate']},
 {'substrates': ['D-ribose 5-phosphate', 'D-xylulose 5-phosphate'],
  'products': ['D-glyceraldehyde 3-phosphate', 'sedoheptulose 7-phosphate']},
 {'substrates': ['D-glyceraldehyde 3-phosphate', 'sedoheptulose 7-phosphate'],
  'products': ['D-ribose 5-phosphate', 'D-xylulose 5-phosphate']}]

In [32]:
substrate = 'D-xylulose 5-phosphate'
res = r.getKMvalues()
[KM['value'] for KM in res[substrate]] 

[0.038,
 0.015,
 0.02,
 0.065,
 0.033,
 0.034,
 0.0056,
 0.067,
 0.23,
 0.255,
 0.303,
 0.12,
 0.16,
 0.1,
 0.18,
 0.41,
 0.04,
 0.49,
 0.011,
 0.2,
 0.15,
 0.44,
 0.0399,
 0.4579,
 0.025,
 0.21,
 0.5,
 0.021,
 0.07,
 0.023,
 0.075,
 0.073,
 1.01,
 0.115,
 0.163,
 1.24,
 0.773,
 3.35,
 0.595,
 4.08,
 0.318,
 0.829,
 0.4]